<a href="https://colab.research.google.com/github/ta269uec/semantic-search/blob/main/k_means_sentence_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing all the packages necessary + loading the data

In [7]:
#!pip install datasets

In [8]:
#!pip install transformers torch

In [9]:
#!pip install sentence-transformers

In [10]:
#!pip install accelerate

In [11]:
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2', device="cuda")

In [41]:
from datasets import list_datasets, load_dataset
def get_reviews_sst2():
  dataset = load_dataset("sst2", split="train")
  negative_reviews = dataset.filter(lambda x:x['label']==0)
  positive_reviews = dataset.filter(lambda x:x['label']==1)
  negative_sentences = [negative_reviews[i]['sentence'] for i in range(len(negative_reviews))]
  positive_sentences = [positive_reviews[i]['sentence'] for i in range(len(positive_reviews))]
  return positive_sentences, negative_sentences

In [42]:
positive_sentences, negative_sentences = get_reviews_sst2()

In [43]:
positive_embeddings, negative_embeddings = embedder.encode(positive_sentences), embedder.encode(negative_sentences)

In [45]:
from sklearn.cluster import KMeans
def cluster_k_means(corpus, corpus_embeddings, num_clusters):
  clustering_model = KMeans(n_clusters=num_clusters)
  clustering_model.fit(corpus_embeddings)
  cluster_assignment = clustering_model.labels_
  clustered_sentences = [[] for i in range(num_clusters)]
  clustered_embeddings = [[] for i in range(num_clusters)]
  for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(corpus[sentence_id])
    clustered_embeddings[cluster_id].append(corpus_embeddings[sentence_id])
  return clustered_sentences, clustered_embeddings

In [48]:
N = 5
clustered_pos_sentences, clustered_pos_embeddings = cluster_k_means(corpus = positive_sentences,\
                                                            corpus_embeddings = positive_embeddings,\
                                                            num_clusters = N)
clustered_neg_sentences, clustered_neg_embeddings = cluster_k_means(corpus = negative_sentences,\
                                                            corpus_embeddings = negative_embeddings,\
                                                            num_clusters = N)

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
